In [ ]:
import os
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import glob
import pandas as pd

%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from matplotlib import gridspec
from scipy.signal import correlate, correlation_lags
from scipy.stats import pearsonr

In [ ]:
# edge_folder = '/gpfs/scratch1/shared/amftrackflow/Drp_downs/20220118_Plate537/20220118_Plate537_005/Analysis/edge (7, 6)/'
vid_folder = "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230123_Plate530/20230123_Plate530_019/"

edge_folders = glob.glob(os.path.join(vid_folder, "Analysis/*)"))
edge_names = []
for i, folder in enumerate(edge_folders):
    edge_name = folder.split("/")[-1][5:]
    print(f"{i}: {edge_name}")

## Here is where you put in the image parameters and the edges you want to see

In [ ]:
edge_interests = [0, 1, 2]
fps = 20
magnification = 50
binning = 2
space_pixel_size = 2 * 1.725 / (magnification) * binning
time_pixel_size = 1 / fps

For Kirchhoff law analysis, we need to assemble some graph arrays.

Here we list out all the nodes, and find their junctions. This results in a dictionary with first a junction as key, its branching nodes as entries, and the flow direction relative to the junction as entry to those. Flow direction is determined by the naming scheme of the edge itself. Up until now, this is a reliable way to determine flow relative to a junction.

In [ ]:
junctions = {}

for i in edge_interests:
    edge_folder = edge_folders[i]
    edge_name = edge_folder.split("/")[-1][5:][1:-1].split(", ")
    edge_array = [int(node) for node in edge_name]
    if edge_array[0] not in junctions:
        junctions[edge_array[0]] = [[edge_array[1], -1]]
    else:
        junctions[edge_array[0]].append([edge_array[1], -1])

    if edge_array[1] not in junctions:
        junctions[edge_array[1]] = [[edge_array[0], -2]]
    else:
        junctions[edge_array[1]].append([edge_array[0], -2])

nodes = [key for key in junctions]
for node in nodes:
    if len(junctions[node]) == 1:
        del junctions[node]
    else:
        junctions[node] = np.array(junctions[node])

print(junctions)

The next batch of code extracts all the transport information from the multiple bin sets we've made. This is put into an array for further junction analysis

In [ ]:
all_transport_plots = {}
width_ratios = []
speed_lefts = []
speed_rights = []
flux_tots = []
edge_names = []

for k, i in enumerate(edge_interests):
    edge_folder = edge_folders[i]
    img_max = 100
    edge_name = edge_folder.split("/")[-1][5:]
    edge_names.append(edge_name)
    speeds_address = glob.glob(edge_folder + "/*speeds*.tiff")[0]
    [speed_left, speed_right, flux_tot] = imageio.imread(speeds_address)
    speed_lefts.append(speed_left)
    speed_rights.append(speed_right)
    flux_tots.append(flux_tot)
    width_ratios.append(len(flux_tot[0]))

fig, ax = plt.subplots(1, 3, figsize=(15, 5), width_ratios=width_ratios)
times = np.linspace(0, time_pixel_size * flux_tots[0].shape[0], flux_tots[0].shape[0])

for i in range(3):
    imshow_extent = [
        0,
        space_pixel_size * flux_tots[i].shape[1],
        time_pixel_size * flux_tots[i].shape[0],
        0,
    ]
    ax[i].imshow(
        flux_tots[i], vmin=-300, vmax=300, cmap="coolwarm", extent=imshow_extent
    )
    ax[i].set_title(f"{edge_names[i]} flux")
fig.tight_layout()

net_speed_plots = np.array(
    [
        np.nansum(flux_tots[i], axis=1) / (space_pixel_size * flux_tots[i].shape[1])
        for i in range(3)
    ]
)
net_speed_max = np.nanmax(abs(net_speed_plots.flatten()))
net_speed_plots /= net_speed_max

cor_lags = correlation_lags(flux_tots[i].shape[0], flux_tots[i].shape[0])
cor_offsets = []
ticknames = ["", "", ""]

fig, ax = plt.subplots(1, 4, figsize=(15, 4))
for i in range(3):
    ticknames[i] = f"{edge_names[i]}\n{edge_names[(i+1)%3]}"
    cor_graph = correlate(net_speed_plots[i], net_speed_plots[(i + 1) % 3])
    pears_graph, sig0 = pearsonr(
        np.cumsum(net_speed_plots[i]), np.cumsum(net_speed_plots[(i + 1) % 3])
    )
    cor_offset = cor_lags[np.argmax(abs(cor_graph))]

    #     print(time_pixel_size * cor_lags[np.argmax(abs(cor_graph))])
    cor_offsets.append(cor_offset)
    if cor_offset > 0:
        pears_graph_offset, sigmax = pearsonr(
            np.cumsum(net_speed_plots[i][cor_offset:]),
            np.cumsum(net_speed_plots[(i + 1) % 3][:-cor_offset]),
        )
    elif cor_offset < 0:
        pears_graph_offset, sigmax = pearsonr(
            np.cumsum(net_speed_plots[i][:cor_offset]),
            np.cumsum(net_speed_plots[(i + 1) % 3][-cor_offset:]),
        )
    else:
        pears_graph_offset = pears_graph
        sigmax = sig0

    ax[0].plot(times, net_speed_plots[i], label=f"{edge_names[i]}")
    ax[1].plot(
        time_pixel_size * cor_lags,
        cor_graph,
        label=f"{edge_names[i]} and {edge_names[(i+1)%3]}",
    )
    ax[2].bar(i, pears_graph, label=f"p={sig0:.3}")
    ax[3].bar(
        i,
        pears_graph_offset,
        label=f"t={cor_offsets[i]*time_pixel_size:.3}\np={sigmax:.3}",
    )
#     print(f"Correlation between {i} and {(i+1)%3}: {correlate(net_speed_plots[i], net_speed_plots[(i+1)%3])}")

cor_offsets = time_pixel_size * np.array(cor_offsets)

ax[0].grid(True)
ax[1].grid(True)
ax[0].set_title("Flux graphs")
ax[1].set_title("Correlation graph")
ax[2].set_title("Pearson Correlation at t=0")
ax[2].set_ylim([-1, 1])
ax[3].set_ylim([-1, 1])
ax[3].set_title("Pearson Correlation at max correlation")
ax[2].set_xticks([0, 1, 2])
ax[2].set_xticklabels(ticknames)
ax[3].set_xticks([0, 1, 2])
ax[3].set_xticklabels(ticknames)
ax[0].legend()
ax[1].legend()
ax[2].legend()
ax[3].legend()

for i in range(4):
    ax[i].axhline(c="black")

fig.tight_layout()